In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import os.path
import json

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
""" if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES) """
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=4000)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

service = None
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

except HttpError as error:
# TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

def getEmailList():
    try:
        results = service.users().messages().list(userId="me").execute()
        messages = results.get("messages", [])

        if not messages:
            print("No messages found.")
            return
        print("Message:")
        for message in messages:
            print(message)

    except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")
    return messages

print(getEmailList())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=915927531429-jb5jkim9325sqr9bun1hvo1evtc961rq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A4000%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=v10Q4eBCzpVAk8sODcw6hSQIFlBlAJ&access_type=offline
Message:
{'id': '18dd72f55c607283', 'threadId': '18dd72e76463d781'}
{'id': '18dd239871ed6fa1', 'threadId': '18dd239871ed6fa1'}
{'id': '18dd235287084801', 'threadId': '18dd235287084801'}
{'id': '18dd2323d21ef2e0', 'threadId': '18dd2323d21ef2e0'}
{'id': '18dceb9bf5ba088a', 'threadId': '18dceb9bf5ba088a'}
{'id': '18dceb8d9d3cdc2a', 'threadId': '18dceb8d9d3cdc2a'}
{'id': '18dceb8d9195dbab', 'threadId': '18dceb8d9195dbab'}
{'id': '18d905ba517a8d6d', 'threadId': '18d905ba517a8d6d'}
{'id': '18d4706edcf6ec88', 'threadId': '18d4706edcf6ec88'}
{'id': '18d016871b43ccaa', 'threadId': '18d016871b43ccaa'}
[{'id': '18dd72f55c607283', 'threadId': 

In [6]:
# Creates a new assistant
assistant = client.beta.assistants.create(
    name = "Gmail Helper",
    instructions = "Your go-to assistant for reading and editing emails. Gmail Helper is a personal assistant dedicated to reading and sending emails, primarily through Google Gmail API integration. Its core functionality includes checking the user’s email to read and summarize emails they receive. It is capable of sending emails to the user's contacts but will always seek confirmation from the user before doing so, ensuring that the user has control over what they send. Gmail Helper maintains a professional demeanor, focusing on providing clear, concise, and accurate scheduling information. It prioritizes user privacy and confidentiality.The assistant is also equipped to guide users through the Google Gmail API’s functionalities and troubleshoot common issues, using language tailored to the user’s level of technical expertise. Overall, Gmail Helper adopts a polite, efficient, and accommodating tone, aiming to make email management and event scheduling as smooth and user-friendly as possible.",
    tools = [{
      "type": "function",
      "function": {
        "name": "getEmailList",
        "description": "Get a list of the user's mailbox contents",
      }
    }],
    model = "gpt-4"
)

In [28]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_H51rILrueFrlL1E4CuX1qqYM', created_at=1708977155, metadata={}, object='thread')


In [29]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "Show me my emails."
)

In [30]:
print(message)

ThreadMessage(id='msg_XphCAaYoc61AeqelrpZbsc7Z', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Show me my emails.'), type='text')], created_at=1708977156, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_H51rILrueFrlL1E4CuX1qqYM')


In [31]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

In [32]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [33]:
status = run.status
print('Current status: ' + status)
if status == 'completed':
    #clearInterval(polling_interval)
    messages_list = assistant.beta.threads.messages.list(thread)
    messages = [message.content for message in messages_list.body.data]
    #res.json({"messages": messages})
elif status == 'requires_action':
    print('requires_action.. looking for a function')
    if run.required_action.type == 'submit_tool_outputs':
        print('submit tool outputs ... ')
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        """ parsed_args = json.loads(tool_calls[0].function.arguments)
        print('Query to search for: ' + parsed_args['query']) """
        api_response = getEmailList()
        run = client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=[{"tool_call_id": tool_calls[0].id, "output": json.dumps(api_response)}])
        print('Run after submit tool outputs: ' + run.status)

Current status: requires_action
requires_action.. looking for a function
submit tool outputs ... 
Message:
{'id': '18dd72f55c607283', 'threadId': '18dd72e76463d781'}
{'id': '18dd239871ed6fa1', 'threadId': '18dd239871ed6fa1'}
{'id': '18dd235287084801', 'threadId': '18dd235287084801'}
{'id': '18dd2323d21ef2e0', 'threadId': '18dd2323d21ef2e0'}
{'id': '18dceb9bf5ba088a', 'threadId': '18dceb9bf5ba088a'}
{'id': '18dceb8d9d3cdc2a', 'threadId': '18dceb8d9d3cdc2a'}
{'id': '18dceb8d9195dbab', 'threadId': '18dceb8d9195dbab'}
{'id': '18d905ba517a8d6d', 'threadId': '18d905ba517a8d6d'}
{'id': '18d4706edcf6ec88', 'threadId': '18d4706edcf6ec88'}
{'id': '18d016871b43ccaa', 'threadId': '18d016871b43ccaa'}
Run after submit tool outputs: queued


In [13]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [36]:
for message in reversed(messages.data):
    print(message.role + ': ' + message.content[0].text.value)

queued
ThreadMessage(id='msg_67cIhzyHD5zWKC8x76TVXFX3', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Show me my emails.'), type='text')], created_at=1708976936, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_ZHtSoxypgX9Za77dLt5aKQbg')
queued
ThreadMessage(id='msg_9k8vltjzylpuG3fbanQWXCmU', assistant_id='asst_ywXPejsKqWHsYi1Y6Rl7QsWe', content=[], created_at=1708976945, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_8hC4KJ7Icf0Na8B4JkAS0i8A', thread_id='thread_ZHtSoxypgX9Za77dLt5aKQbg')


In [ ]:
service.close()